In [18]:
import random

import pandas as pd

from tqdm.auto import tqdm

In [19]:
ratings = pd.read_csv('/pio/scratch/1/i313924/slim_ratings.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'])
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [20]:
popular_items = ratings['asin'].unique()

In [23]:
# jaki rzad wielkości tego pliku?
def prepare_test_file(categories, path, test_set_size = 10000,
                      category_type = "category_1", weighted_sample = True,
                      min_counts = 0, popular_items = None):
    
    counts = categories[category_type].value_counts()
    counts = counts[counts > min_counts] 
    categories_grouped = categories.groupby(category_type)
    
    categories_dict = {}
    for category, items in categories_grouped:
        if popular_items is not None:
            categories_dict[category] = items.loc[items['asin'].isin(popular_items)]
        else:
            categories_dict[category] = items                                         

    test_set = []    
    for _ in tqdm(range(test_set_size)):
        
        # choosing categories for ABX
        if weighted_sample:
            category_sample = counts.sample(2, weights=counts)
        else:
            category_sample = counts.sample(2)
            
        # choosing which category to treat as positive & negative    
        positive = category_sample.index[0]
        negative = category_sample.index[1]
        if random.random() < 0.5:
            positive, negative = negative, positive
            
        # chosing items for ABX from positive & nega    
        positive_items = categories_dict[positive].sample(2).asin.values
        negative_item = categories_dict[negative].sample(1).asin.values
        
        # appending record
        line = {"A": positive_items[0],
                "B": negative_item[0],
                "X": positive_items[1],
                "category_AX": positive,
                "category_B": negative}
        test_set.append(line)
        
    with open(path, 'w') as file:    
        pd.DataFrame(test_set).to_json(file, 'records', lines = True)

In [24]:
prepare_test_file(categories, '/pio/scratch/1/i313924/test_data/test_ABX_slim', category_type='category_1', test_set_size = 10000, popular_items = popular_items)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [92]:
counts = categories["category_2"].value_counts()
counts

Clothing                                                                1044952
Shoes                                                                    466895
Jewelry                                                                  399823
Accessories                                                              186716
Watches                                                                  123775
                                                                         ...   
Size: 0.20 x inch                                                             1
Size details ,please see the size table pictures with tshirt photoes          1
Material: Copper, zircon                                                      1
Soft woven canvas fabric                                                      1
                                                                              1
Name: category_2, Length: 9908, dtype: int64